# pyscopg2でPythonからPostgreSQLにつないでみよう！

## 自己紹介

* 株式会社アシスト勤務
* DBエンジニア
* PostgreSQL / Oracle 担当
* Python歴3年?



## なんでこんなことに

* stapy参加4回目
* 毎回LTで適当なこと喋ってた

http://www.slideshare.net/satoshiyamada71697
<table>
<tr>
<td>
<img src="./img/psycopg1.png" width='250px'/>
</td>
<td>
<img src="./img/psycopg2.png" width='250px'/>
</td>
</tr>
</table>



ら、阿久津さんに目をつけられて本番でやれと。

### アジェンダ

1. PostgreSQL
2. psycopg2?
3. psycopg2のインストール
4. 基本的な使い方
5. ちょっとお得な使い方

## PostgreSQL?
![](./img/postgres.png)

## 雑なPostgreSQLの説明

* PostgreaSQLはOSSのRDBMS
* MySQLと同じで無料で使える!
* 有償サポートをしている会社も結構(弊社とか)
* MySQLに比べると国内知名度は少し下？
* でもMySQLは●racleの政策でちょっと先行き不安
* だから、PostgreSQLもっとつかおうず！

### PostgreSQLのいいところ

* アーキテクチャがシンプルな割に機能が豊富(●racleは凄いけと中が複雑で。。)
* レプリケーションも簡単
* **json**のサポートが手厚い


#### PostgreSQL動かしたい！

[公式](http://www.postgresql.org/download/)から自分の環境にあったものを入手してください。  

最近だとdockerやVagrantでもPostgreSQLが入ったイメージがあるので、そちらでもいいでしょう。

# psycopg2
ロゴがほんとにサイコっぽい・・・
![](./img/psycopg2_logo.png)

## Pyscopg2て？

* PythonからPostgreSQLにつなぐためのアダプター
* PythonからDBを操作する仕様DB-API 2.0準拠
* デファクトに近い？(Djangoもdjango.db.backends.postgresql_psycopg2で使う)
* Python2.5-2.7,3.1-3.4まで対応
* 接続先のPostgreSQLも現在のほぼすべてのリリース対応(7.4〜)

# psycopg2のインストール方法


普通のパッケージと同じように

```
pip install psycopg2
```

で動かせることも多いですが、psycopg2はlibpqというPostgreSQLのCのライブラリに  
依存しているので、環境によってはうまく動かないかもしれません。

特に、Unix系の環境ではlibpqライブラリの参照でエラーになるケースがあります。  
その場合はlibpq-dev python-devあたりを入れてみてください。

なお、基本的なプラットフォームにはビルド済のパッケージがあるので  
そちらを利用するほうが簡単かもしれません。


#### Windows

[ここ](http://www.stickpeople.com/projects/python/win-psycopg/) から対応するインストーラをDLしましょう。  
自分が使っているPythonのバージョンやBit数に気をつけてください。

#### Linux(Ubuntu等のDebian系)

ビルドされたパッケージがリポジトリにあります。

```
sudo apt-get install python-psycopg2
```


#### Mac

##### finkを使う場合
```
fink install psycopg2-py27
```

##### MacPortsを使う場合
```
sudo port install py27-psycopg2
```


# 基本的な使い方

だいたいこんな感じで動かすことができます。以下は**localhost:9450**で起動している  
**sampledb**というDBへ、**denzow**というユーザ名で**passw0rd**をパスワードとして  
接続して処理をするコードです。

In [25]:
# psycopg2を使えるようにして
import psycopg2
# 接続して
conn = psycopg2.connect("host=localhost port=9450 dbname=sampledb user=denzow password=passw0rd")
# SQLを投げるためのカーソル作って
cur = conn.cursor()
# SQLを発行
cur.execute("select * from emp where deptno=%s", (10,))
# 結果を表示
for row in cur:
    print(row)
# COMMIT!
conn.commit()

(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450, None, 10)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000, None, 10)
(7934, 'MILLER', 'CLERK', 7782, datetime.datetime(1982, 1, 23, 0, 0), 1300, None, 10)


おそらく、会場の99%の方はローカルにPostgreSQLは入っていないでしょう。それでも手元で
コードを試したい場合は、以下を実行してから試してください。  
PostgreSQLの代わりにsqlite3のを使用するためのコードです。


In [88]:
# postgresqlが用意できない環境ではsqliteを使ってデモを動かします
import sqlite3
# sqliteは指定したパスにDBファイルを作成して利用します
connection = sqlite3.connect("./tmp_sqlite.db")
# サンプルとして某EMP・DEPT表をセットアップします。
try:
    connection.executescript("""
    drop table emp;
    drop table dept;
    """)
except:
    connection.rollback()
connection.executescript("""
CREATE TABLE EMP
       (EMPNO INT NOT NULL,
        ENAME VARCHAR(10),
        JOB VARCHAR(9),
        MGR INT,
        HIREDATE TIMESTAMP,
        SAL INT,
        COMM INT,
        DEPTNO INT);

INSERT INTO EMP VALUES
        (7369, 'SMITH',  'CLERK',     7902,
        '17-DEC-1980',  800, NULL, 20);
INSERT INTO EMP VALUES
        (7499, 'ALLEN',  'SALESMAN',  7698,
        '20-FEB-1981', 1600,  300, 30);
INSERT INTO EMP VALUES
        (7521, 'WARD',   'SALESMAN',  7698,
        '22-FEB-1981', 1250,  500, 30);
INSERT INTO EMP VALUES
        (7566, 'JONES',  'MANAGER',   7839,
        '2-APR-1981',  2975, NULL, 20);
INSERT INTO EMP VALUES
        (7654, 'MARTIN', 'SALESMAN',  7698,
        '28-SEP-1981', 1250, 1400, 30);
INSERT INTO EMP VALUES
        (7698, 'BLAKE',  'MANAGER',   7839,
        '1-MAY-1981',  2850, NULL, 30);
INSERT INTO EMP VALUES
        (7782, 'CLARK',  'MANAGER',   7839,
        '9-JUN-1981',  2450, NULL, 10);
INSERT INTO EMP VALUES
        (7788, 'SCOTT',  'ANALYST',   7566,
        '09-DEC-1982', 3000, NULL, 20);
INSERT INTO EMP VALUES
        (7839, 'KING',   'PRESIDENT', NULL,
        '17-NOV-1981', 5000, NULL, 10);
INSERT INTO EMP VALUES
        (7844, 'TURNER', 'SALESMAN',  7698,
        '8-SEP-1981',  1500,    0, 30);
INSERT INTO EMP VALUES
        (7876, 'ADAMS',  'CLERK',     7788,
        '12-JAN-1983', 1100, NULL, 20);
INSERT INTO EMP VALUES
        (7900, 'JAMES',  'CLERK',     7698,
        '3-DEC-1981',   950, NULL, 30);
INSERT INTO EMP VALUES
        (7902, 'FORD',   'ANALYST',   7566,
        '3-DEC-1981',  3000, NULL, 20);
INSERT INTO EMP VALUES
        (7934, 'MILLER', 'CLERK',     7782,
        '23-JAN-1982', 1300, NULL, 10);

CREATE TABLE DEPT
       (DEPTNO INT,
        DNAME VARCHAR(14),
        LOC VARCHAR(13));

INSERT INTO DEPT VALUES (10, 'ACCOUNTING', 'NEW YORK');
INSERT INTO DEPT VALUES (20, 'RESEARCH',   'DALLAS');
INSERT INTO DEPT VALUES (30, 'SALES',      'CHICAGO');
INSERT INTO DEPT VALUES (40, 'OPERATIONS', 'BOSTON');
""")
print(connection)

PostgreSQLが用意出来ている場合は、以下を実行してください。

In [91]:
import psycopg2 
# 接続情報は書き換えてください
connection = psycopg2.connect("host=localhost port=9450 dbname=sampledb user=denzow password=passw0rd")
cur = connection.cursor()
try:
    cur.execute("""
    drop table emp;
    drop table dept;
    """)
except Exception as e:
    print(e)
    connection.rollback()
    
cur.execute("""
CREATE TABLE EMP
       (EMPNO INT NOT NULL,
        ENAME VARCHAR(10),
        JOB VARCHAR(9),
        MGR INT,
        HIREDATE TIMESTAMP,
        SAL INT,
        COMM INT,
        DEPTNO INT);
INSERT INTO EMP VALUES
        (7369, 'SMITH',  'CLERK',     7902,
        TO_DATE('17-DEC-1980', 'DD-MON-YYYY'),  800, NULL, 20);
INSERT INTO EMP VALUES
        (7499, 'ALLEN',  'SALESMAN',  7698,
        TO_DATE('20-FEB-1981', 'DD-MON-YYYY'), 1600,  300, 30);
INSERT INTO EMP VALUES
        (7521, 'WARD',   'SALESMAN',  7698,
        TO_DATE('22-FEB-1981', 'DD-MON-YYYY'), 1250,  500, 30);
INSERT INTO EMP VALUES
        (7566, 'JONES',  'MANAGER',   7839,
        TO_DATE('2-APR-1981', 'DD-MON-YYYY'),  2975, NULL, 20);
INSERT INTO EMP VALUES
        (7654, 'MARTIN', 'SALESMAN',  7698,
        TO_DATE('28-SEP-1981', 'DD-MON-YYYY'), 1250, 1400, 30);
INSERT INTO EMP VALUES
        (7698, 'BLAKE',  'MANAGER',   7839,
        TO_DATE('1-MAY-1981', 'DD-MON-YYYY'),  2850, NULL, 30);
INSERT INTO EMP VALUES
        (7782, 'CLARK',  'MANAGER',   7839,
        TO_DATE('9-JUN-1981', 'DD-MON-YYYY'),  2450, NULL, 10);
INSERT INTO EMP VALUES
        (7788, 'SCOTT',  'ANALYST',   7566,
        TO_DATE('09-DEC-1982', 'DD-MON-YYYY'), 3000, NULL, 20);
INSERT INTO EMP VALUES
        (7839, 'KING',   'PRESIDENT', NULL,
        TO_DATE('17-NOV-1981', 'DD-MON-YYYY'), 5000, NULL, 10);
INSERT INTO EMP VALUES
        (7844, 'TURNER', 'SALESMAN',  7698,
        TO_DATE('8-SEP-1981', 'DD-MON-YYYY'),  1500,    0, 30);
INSERT INTO EMP VALUES
        (7876, 'ADAMS',  'CLERK',     7788,
        TO_DATE('12-JAN-1983', 'DD-MON-YYYY'), 1100, NULL, 20);
INSERT INTO EMP VALUES
        (7900, 'JAMES',  'CLERK',     7698,
        TO_DATE('3-DEC-1981', 'DD-MON-YYYY'),   950, NULL, 30);
INSERT INTO EMP VALUES
        (7902, 'FORD',   'ANALYST',   7566,
        TO_DATE('3-DEC-1981', 'DD-MON-YYYY'),  3000, NULL, 20);
INSERT INTO EMP VALUES
        (7934, 'MILLER', 'CLERK',     7782,
        TO_DATE('23-JAN-1982', 'DD-MON-YYYY'), 1300, NULL, 10);

CREATE TABLE DEPT
       (DEPTNO INT,
        DNAME VARCHAR(14),
        LOC VARCHAR(13));

INSERT INTO DEPT VALUES (10, 'ACCOUNTING', 'NEW YORK');
INSERT INTO DEPT VALUES (20, 'RESEARCH',   'DALLAS');
INSERT INTO DEPT VALUES (30, 'SALES',      'CHICAGO');
INSERT INTO DEPT VALUES (40, 'OPERATIONS', 'BOSTON');
""")
connection.commit()
print(connection)

<connection object at 0x7f16e41a27b8; dsn: 'host=localhost port=9450 dbname=sampledb user=denzow password=xxxxxxxx', closed: 0>


#### 接続について(connect)

以下を**connect()**の引数として指定できます。connectの引数は

> connect("DB接続文字列")

という形式で文字列(DSN文字列)として情報を指定する方法と

> connect(database="...", host="",...)

といった名前付きパラメータとして渡す2種類があります。

|引数名|意味|
|----|------|
|dbname|接続DB名。DSN文字列の時に使う|
|database|接続DB名。名前付きパラメータの時に使う|
|user|接続に使用するDBユーザ名|
|password|接続に使用するDBユーザ名のパスワード|
|host|接続先のアドレス。省略時はUNIXドメインソケットでの通信を試みる|
|port|接続先のポート番号。省略時はPostgreSQL標準の5432が指定されたものとして動く|


### カーソルについて(cursor())

カーソルはSQLを発行するために使用します。**connection.cursor()**で取得することができます。使い終わったらcloseをコールします。

In [82]:
cursor1 = connection.cursor()
# なにかSQLの処理をcursor1で行う
cursor1.close()

cursorオブジェクトのexecuteメソッドでSQLを発行することができます。JDBCのexecute/executeQueryとは違い
SELCT/DMLのどちらであってもexecuteが使用できます。  

結果を取得する場合は、cursorオブジェクトの**fetchone/fetchmany**メソッドが使用できます

In [83]:
# sqlite is ok

cursor1 = connection.cursor()
cursor1.execute("select empno,ename from emp")
# 1行目
print(cursor1.fetchone())
# 2行目
print(cursor1.fetchone())
# 3行目
print(cursor1.fetchone())
# 残りの行を一気に(20行を指定しているが、残りは11行なのでそこだけ戻る)
print(cursor1.fetchmany(20))
# もうないので空のリストが戻る
print(cursor1.fetchmany(20))

# 後処理
cursor1.close()
connection.commit()

(7369, 'SMITH')
(7499, 'ALLEN')
(7521, 'WARD')
[(7566, 'JONES'), (7654, 'MARTIN'), (7698, 'BLAKE'), (7782, 'CLARK'), (7788, 'SCOTT'), (7839, 'KING'), (7844, 'TURNER'), (7876, 'ADAMS'), (7900, 'JAMES'), (7902, 'FORD'), (7934, 'MILLER')]
[]


カーソルオブジェクトは**iterable**なので、そのままfor文にかけても問題ありません。



In [84]:
cursor1 = connection.cursor()
cursor1.execute("select empno,ename from emp")
for row in cursor1:
    print(row)

# 後処理
connection.commit()
cursor1.close()

(7369, 'SMITH')
(7499, 'ALLEN')
(7521, 'WARD')
(7566, 'JONES')
(7654, 'MARTIN')
(7698, 'BLAKE')
(7782, 'CLARK')
(7788, 'SCOTT')
(7839, 'KING')
(7844, 'TURNER')
(7876, 'ADAMS')
(7900, 'JAMES')
(7902, 'FORD')
(7934, 'MILLER')


### 結果セットについて

* デフォルトではタプルで結果が戻る
* 正直列名をキーにしたディクショナリのほうが嬉しい時もある
    - タプルだとインデックスでのアクセス
    - あとで何列目がなんのデータか忘れる
    - ディクショナリならあとで見てもわかりやすい
    
そんな時には**DictCursor**が便利です。

In [85]:
# not work in sqlite.
import psycopg2.extras
# di
dict_cur = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
dict_cur.execute("select empno,ename from emp")
for row in dict_cur:
    # 列名をキーとしてアクセスできる
    print("empno is", row["empno"], "ename is", row["ename"])

connection.commit()
dict_cur.close()

empno is 7369 ename is SMITH
empno is 7499 ename is ALLEN
empno is 7521 ename is WARD
empno is 7566 ename is JONES
empno is 7654 ename is MARTIN
empno is 7698 ename is BLAKE
empno is 7782 ename is CLARK
empno is 7788 ename is SCOTT
empno is 7839 ename is KING
empno is 7844 ename is TURNER
empno is 7876 ename is ADAMS
empno is 7900 ename is JAMES
empno is 7902 ename is FORD
empno is 7934 ename is MILLER


なお、pysocpg2.connection()実行時に**cursor_factory**パラメータにDictCursorを指定するとそのconnectionから
取得されるカーソルはDictCursorになります。

### 値のバインドについて

Where句だけ違うSQLを発行することは多々あります。そんな時には一部だけ値を入れ替えて実行したくなります。
psycopg2では**%s**をプレースホルダに指定します。以下はdeptnoが10、20のそれぞれのSQLを処理しています。

In [97]:
# not work in sqlite
# sqlite は%sではなく?をプレースホルダーに使用する
try:
    cursor1 = connection.cursor()
    # deptno=の箇所だけ変数化
    base_sql = "select empno,ename from emp where deptno=%s"
    print("##DEPTNO=10 is.")
    cursor1.execute(base_sql, (10,))
    for row in cursor1:
        print(row)

    print("##DEPTNO=20 is.")
    cursor1.execute(base_sql, (20,))
    for row in cursor1:
        print(row)
finally:
    # 後処理
    connection.commit()
    cursor1.close()

##DEPTNO=10 is.
(7782, 'CLARK')
(7839, 'KING')
(7934, 'MILLER')
##DEPTNO=20 is.
(7369, 'SMITH')
(7566, 'JONES')
(7788, 'SCOTT')
(7876, 'ADAMS')
(7902, 'FORD')


In [100]:
# not work in sqlite
# sqlite は%sではなく?をプレースホルダーに使用する
# 名前付きプレースホルダの例
try:
    cursor1 = connection.cursor()
    # deptno=の箇所だけ変数化
    base_sql = "select empno,ename from emp where deptno=%(dept_no)s"
    print("##DEPTNO=10 is.")
    cursor1.execute(base_sql, {"dept_no":10})
    for row in cursor1:
        print(row)

    print("##DEPTNO=20 is.")
    cursor1.execute(base_sql, {"dept_no":20})
    for row in cursor1:
        print(row)
finally:
    # 後処理
    connection.commit()
    cursor1.close()

##DEPTNO=10 is.
(7782, 'CLARK')
(7839, 'KING')
(7934, 'MILLER')
##DEPTNO=20 is.
(7369, 'SMITH')
(7566, 'JONES')
(7788, 'SCOTT')
(7876, 'ADAMS')
(7902, 'FORD')
